In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

# strainforfard peaks search
import plotly.graph_objects as go
from scipy.signal import find_peaks, savgol_filter

# peaks search by gauss decomposition
import gausspy
import gausspy.gp as gp
import pickle

NameError: name 'y_fg' is not defined

In [ ]:
# ga_file = "/content/drive/MyDrive/AIRI/RamanSpectra/ga_2_3_5_6.csv"
# fg_file = "/content/drive/MyDrive/AIRI/RamanSpectra/fg_1-5_7-11.csv"
ga_file = "dataSrc/ga_2_3_5_6.csv"
fg_file = "dataSrc/fg_1-5_7-11.csv"
data_ga = pd.read_csv(ga_file, sep=";")
print(f"data with GA samples shape: {data_ga.shape}")
data_fg = pd.read_csv(fg_file, sep=";")
print(f"data with FG samples shape: {data_fg.shape}")

In [ ]:
y_fg = data_fg['class'].values
X_fg = data_fg.drop(columns=['class'])
print(f"shapes of X and y of FG samples respectively is: {X_fg.shape}, {y_fg.shape}")

y_ga = data_ga['class'].values
X_ga = data_ga.drop(columns=['class'])
print(f"shapes of X and y of GA samples respectively is: {X_ga.shape}, {y_ga.shape}")

In [ ]:
print(f"old fg classes: {set(y_fg)}")
for index, element in enumerate(set(y_fg)):
    y_fg[y_fg == element] = index
print(f"new fg classes: {set(y_fg)}\n")

print(f"old ga classes: {set(y_ga)}")
for index, element in enumerate(set(y_ga)):
    y_ga[y_ga == element] = index + len(set(y_fg))
print(f"new ga classes: {set(y_ga)}")

In [ ]:
group1 = [0, 1, 2, 3, 4]
group2 = [5, 6, 7, 8, 9]
group3 = [10, 11]
group4 = [12, 13]
groups = [group1, group2, group3, group4]

def add_group(y, groups, add=0):
    new_y = np.zeros((y.shape[0], 2))
    new_y[:, 1] = y
    for group_num, group in enumerate(groups):
        for class_num in group:
            new_y[y==class_num] = np.array([group_num + add, class_num])
    return new_y

y_ga = add_group(y_ga, (group3, group4), add=2)
y_fg = add_group(y_fg, (group1, group2))
print(f"Now, shapes of y_fg and ga is respectively: {y_ga.shape}, {y_fg.shape}")

In [ ]:
y = np.concatenate((y_fg, y_ga), axis=0)
X = np.concatenate((X_fg, X_ga), axis=0)
X = X.reshape(X.shape[0], 1, -1)
print(f"shapes of X and y is respectively: {X.shape}, {y.shape}")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
data_loader_train = DataLoader(tuple(zip(torch.tensor(X_train).float(), torch.tensor(y_train).long())), 
                                      batch_size=16, 
                                      shuffle=True)
data_loader_test = DataLoader(tuple(zip(torch.tensor(X_test).float(), torch.tensor(y_test).long())), 
                                     batch_size=16, 
                                     shuffle=False)